In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Paths

In [ ]:
#nz_goals_fn = '../../data/nz_goals/nz_goals.csv'
nz_goals_fn = snakemake.input.nz_goals

### Define variables

In [ ]:
sweep = "expansion"

In [ ]:
re_share = 88# in percent
gen_total = 10 # in TWh
ely_eff = 0.62 # in percent (https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2030.csv)

rule = 90 # in percent
rule_sweep = np.array([60, 70, 80, 90]) # in percent

re_exp = np.array([1]) # TWh/year
re_exp_sweep = np.array([0,1,2]) # TWh/year

year_target = 2035
year_current = 2024

demand_dom_incr = np.array([0, 5, 10, 20]) # in TWh

In [ ]:
#demand_dom_incr_grid, re_exp_grid = np.meshgrid(demand_dom_incr, re_exp)

### Formula

In [ ]:
def get_export(sweep="expansion", re_exp_sweep=re_exp_sweep, rule_sweep=rule_sweep, rule=rule, re_exp=re_exp, gen_total=gen_total, ely_eff=ely_eff, year_target=year_target, year_current=year_current, demand_dom_incr=demand_dom_incr):

    if sweep == "expansion":
        dim = re_exp_sweep
        demand_dom_incr_grid, re_exp = np.meshgrid(demand_dom_incr, re_exp_sweep)
    elif sweep == "rule":
        dim = rule_sweep
        demand_dom_incr_grid, rule = np.meshgrid(demand_dom_incr, rule_sweep)

    export_el = ((re_share-rule)/100 * gen_total) + (re_exp * (year_target-year_current)) - demand_dom_incr_grid

    export = export_el * ely_eff
    export_df = pd.DataFrame(export, columns=demand_dom_incr, index=dim)
    export_df.index.name = sweep
    
    return export_df

In [ ]:
export_df = get_export(sweep=sweep)

### NZ roadmap

In [ ]:
nz_goals = pd.read_csv(nz_goals_fn, index_col=0) 
nz_goals *= 33.33e-6 # Convert from tons to TWh hydrogen

### Plot

In [ ]:
print(snakemake.output)

In [ ]:
fig = plt.figure(facecolor='white', figsize=(6, 5))

ax = fig.add_subplot(1,1,1)
alpha=1

if sweep == "expansion":
    for re_exp_val in np.unique(re_exp_sweep):
        plt.plot(demand_dom_incr, export_df.loc[re_exp_val], label=f'{re_exp_val} TWh/a', color='green', alpha=alpha)
        alpha -= 0.2

elif sweep == "rule":
    for re_exp_val in np.unique(rule_sweep):
        plt.plot(demand_dom_incr, export_df.loc[re_exp_val], label=f'{re_exp_val}% rule', color='green', alpha=alpha)
        alpha -= 0.2

# Plot nz hydrogen goals (arrows)
# plt.text(2, nz_goals.loc[2035, "high"], 'NZ 2035 goal', color='red', alpha=1, fontsize=8)
# plt.arrow(2, nz_goals.loc[2035, "high"]+0.27, -0.5,0, head_width=1, head_length=0.5, fc='black', ec='black', alpha=0.5)

# Plot nz hydrogen goals (lines)
for year in nz_goals.index:
    plt.axhline(y=nz_goals.loc[year, "high"], color='red', linestyle='--', alpha=0.5)
    plt.text(0.4, nz_goals.loc[year, "high"]+0.3, f'NZ {year} export goal', color='red', alpha=1, fontsize=8)

plt.xlabel('Demand increase in TWh')
plt.ylabel('Export in TWh')
plt.ylim(-5, 35)
plt.xlim(0, 20)
plt.grid()
plt.legend()
#plt.title('Export vs. Demand increase')

fig.savefig(snakemake.output.exportquan, bbox_inches="tight")

plt.show()